In [ ]:
import tkinter as tk
from tkinter import messagebox, ttk
from tkinter import font as tkFont
#from PIL import Image, ImageTk
import random
#import pandas as pd
import os


In [ ]:
# Función para recomendar tecnica de aprendizaje profundo
#todo
def recomendar_deeplearning(raza, fee, mascotas, num_recomendaciones=3):
    mascotas_filtradas = mascotas.query(f"Fee <= {int(fee)}")
    
    #por cada filtro, un if con su query
    if raza:
        mascotas_filtradas = mascotas_filtradas.query(f"Type == {raza}")

    #entregando las top 3 mascotas
    if not mascotas_filtradas.empty:
        mascotas_filtradas = mascotas_filtradas.sort_values(by='Fee', ascending=False)
        return mascotas_filtradas.head(num_recomendaciones)
    else:
        return pd.DataFrame()

In [ ]:
# Función para mostrar la recomendación como widget
def mostrar_rec(recomendacion):
    for widget in frame_resultados.winfo_children():
        widget.destroy()

    for index, pelicula in recomendacion.iterrows():
        img_path = os.path.join('train_images_preprocessed', pelicula['PetID']+'-0.jpg')
        if os.path.exists(img_path):
            img = Image.open(img_path)
        else: #los perros no tienen imagenes guardadas, por lo que se utiliza esta de un gato como placeholder
            img = Image.open(os.path.join('train_images_preprocessed', '002278114-16.jpg'))
        
        #messagebox.showinfo(title='img', message=img)
        
        img = img.resize((180, 180), Image.ANTIALIAS)
        img = ImageTk.PhotoImage(img)
        
        frame = tk.Frame(frame_resultados)
        frame.pack(side='left', padx=10)
        
        label_img = tk.Label(frame, image=img)
        label_img.image = img  # Guardar la referencia
        label_img.pack()
        
        label_text = tk.Label(frame, text=f"{pelicula['Name']}\nAge: {pelicula['Age']} \nDesc: {pelicula['Description'][:150]}", wraplength=180)
        label_text.pack()

In [ ]:
# Función para manejar el botón de recomendación
def recomendar_boton():    
    raza = entry_type.get()
    fee = entry_fee.get()
    
    if not fee:
        fee = "0"
    
    recomendaciones = recomendar_deeplearning(raza, fee, mascotas)

    if not recomendaciones.empty:
        mostrar_rec(recomendaciones)
    else:
        messagebox.showwarning("Sin recomendaciones", f"No se encontraron recomendaciones atingentes.")

In [ ]:
# Crear la ventana principal
root = tk.Tk()
root.geometry("900x800")
root.title("Sistema de recomendación ~ tecnicas deeplearning")
root.configure(bg="#f0f0f5")  # Color de fondo de la ventana principal

# Fuente personalizada
font_label = tkFont.Font(family="Helvetica", size=16)
font_entry = tkFont.Font(family="Helvetica", size=14)
font_button = tkFont.Font(family="Helvetica", size=16, weight="bold")

# Etiqueta para raza
label_type = tk.Label(root, text="Elige la raza preferida de la mascota :", font=font_label, bg="#f0f0f5")
label_type.pack(pady=20)

# Boton radio para raza
entry_type = tk.StringVar()
values = {
    "Perro" : "1",
    "Gato" : "2"
}
for (text, value) in values.items():
    tk.Radiobutton(root, text=text, variable=entry_type, value=value).pack(pady=5)

# Etiqueta para Fee
label_fee = tk.Label(root, text="Cual es el máximo que puede pagar por la mascota?:", font=font_label, bg="#f0f0f5")
label_fee.pack(pady=20)

# Boton check para Fee
entry_fee = tk.Spinbox(root, from_=0, to=3000, font=font_entry, relief="sunken", repeatdelay=500, repeatinterval=10, width=30)
entry_fee.pack(pady=10)

# Botón para recomendar películas
btn_recomendar = tk.Button(root, text="Recomendar Mascota", command=recomendar_mascota, font=font_button, 
                           bg="#4CAF50", fg="white", height=2, width=20)
btn_recomendar.pack(pady=20)

# Frame para los resultados (en la misma ventana)
frame_resultados = tk.Frame(root, bg="#f0f0f5")
frame_resultados.pack(pady=20)

# Etiqueta para mostrar los resultados
label_resultado = tk.Label(frame_resultados, text="", font=font_label, bg="#f0f0f5", fg="#333333")
label_resultado.pack()

# Ejecutar el loop principal de la GUI
root.mainloop()